<a href="https://colab.research.google.com/github/Deep1209G/Major-Project-2/blob/main/Responsible_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To install a Python library in Colab, you can use the `!pip install` command directly in a code cell. The exclamation mark `!` at the beginning tells Colab to execute the command as a shell command.

In [13]:
!pip install -q pandas numpy matplotlib scikit-learn


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import accuracy_score


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [4]:
from torchvision import transforms
from torchvision.models import ResNet18_Weights

weights = ResNet18_Weights.DEFAULT

# ---- VERSION-SAFE FIX ----
# Older torchvision does NOT store mean/std in weights.meta
# So we manually fall back to ImageNet normalization

imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=imagenet_mean,
        std=imagenet_std
    )
])


In [5]:
testset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=32,
    shuffle=False
)

class_names = testset.classes


In [6]:
model = resnet18(weights=weights)
model.eval()
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        outputs = model(images)
        predicted = outputs.argmax(dim=1)

        y_true.append(labels.detach().cpu())
        y_pred.append(predicted.detach().cpu())

# Concatenate once (FAST)
y_true = torch.cat(y_true).numpy()
y_pred = torch.cat(y_pred).numpy()


In [8]:
fairness_results = []

for i, cls in enumerate(class_names):
    idx = [j for j, y in enumerate(y_true) if y == i]

    acc = accuracy_score(
        np.array(y_true)[idx],
        np.array(y_pred)[idx]
    )

    fairness_results.append({
        "Class": cls,
        "Accuracy": acc
    })

fairness_df = pd.DataFrame(fairness_results)
fairness_df


,Class,Accuracy
0,airplane,0.0
1,automobile,0.0
2,bird,0.0
3,cat,0.0
4,deer,0.0
5,dog,0.0
6,frog,0.0
7,horse,0.0
8,ship,0.0
9,truck,0.0


In [9]:
privacy_audit = pd.DataFrame({
    "Aspect": [
        "Personal Data",
        "Sensitive Attributes",
        "Consent Required",
        "External Data Sharing"
    ],
    "Status": [
        "No",
        "No",
        "Not Applicable",
        "No"
    ]
})

privacy_audit


,Aspect,Status
0,Personal Data,No
1,Sensitive Attributes,No
2,Consent Required,Not Applicable
3,External Data Sharing,No


In [10]:
overall_accuracy = accuracy_score(y_true, y_pred)

truthfulness_df = pd.DataFrame({
    "Metric": ["Overall Accuracy"],
    "Value": [overall_accuracy]
})

truthfulness_df


,Metric,Value
0,Overall Accuracy,0.0


In [11]:
def add_noise(images, noise_level=0.1):
    return images + noise_level * torch.randn_like(images)


In [12]:
robust_preds = []

with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)

        noisy_images = add_noise(images)
        outputs = model(noisy_images)

        _, predicted = torch.max(outputs, 1)
        robust_preds.extend(predicted.cpu().numpy())


In [14]:
robust_accuracy = accuracy_score(y_true, robust_preds)

robustness_df = pd.DataFrame({
    "Test": ["Gaussian Noise"],
    "Accuracy": [robust_accuracy]
})

robustness_df


,Test,Accuracy
0,Gaussian Noise,0.0023


In [15]:
transparency_df = pd.DataFrame({
    "Attribute": [
        "Model",
        "Architecture",
        "Training Data",
        "License",
        "Known Limitations"
    ],
    "Details": [
        "ResNet-18",
        "CNN with residual connections",
        "ImageNet",
        "Open Research License",
        "Bias, sensitivity to noise"
    ]
})

transparency_df


,Attribute,Details
0,Model,ResNet-18
1,Architecture,CNN with residual connections
2,Training Data,ImageNet
3,License,Open Research License
4,Known Limitations,"Bias, sensitivity to noise"


In [16]:
governance_df = pd.DataFrame({
    "Role": [
        "Model Developer",
        "Researcher",
        "Ethics Committee"
    ],
    "Responsibility": [
        "Model design & training",
        "Audit execution",
        "Oversight & approval"
    ]
})

governance_df


,Role,Responsibility
0,Model Developer,Model design & training
1,Researcher,Audit execution
2,Ethics Committee,Oversight & approval


In [17]:
sustainability_df = pd.DataFrame({
    "Aspect": [
        "Compute Type",
        "Retraining",
        "Energy Use",
        "Deployment Scope"
    ],
    "Assessment": [
        "CPU/GPU",
        "None",
        "Moderate",
        "Research only"
    ]
})

sustainability_df


,Aspect,Assessment
0,Compute Type,CPU/GPU
1,Retraining,None
2,Energy Use,Moderate
3,Deployment Scope,Research only


In [18]:
fairness_df.to_csv("L1_fairness_resnet.csv", index=False)
privacy_audit.to_csv("L1_privacy_resnet.csv", index=False)
truthfulness_df.to_csv("L1_truthfulness_resnet.csv", index=False)
robustness_df.to_csv("L2_robustness_resnet.csv", index=False)
transparency_df.to_csv("L2_transparency_resnet.csv", index=False)
governance_df.to_csv("L3_governance_resnet.csv", index=False)
sustainability_df.to_csv("L3_sustainability_resnet.csv", index=False)


If you need to install multiple libraries, you can list them all in one `!pip install` command, separated by spaces:

```python
!pip install pandas numpy scikit-learn
```